In [87]:
%pip install lightfm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 310 kB 1.4 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=706166 sha256=f3bb0482d458c317628b7233fc83ec4adcc2b73d04741526bfa64d61c0ce4851
  Stored in directory: /tmp/xdg_cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [89]:
%pip install -q kaggle

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [90]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/home/jupyter/.kaggle’: File exists


Exception: Process exited with code 1

In [91]:
! kaggle datasets download -d grouplens/movielens-20m-dataset

movielens-20m-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
 ! unzip movielens-20m-dataset.zip -d dataset

In [396]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import scipy
from tqdm import tqdm

In [246]:
ratings = pd.read_csv('dataset/rating.csv')
genomes = pd.read_csv('dataset/genome_tags.csv')
movie_props = pd.read_csv('dataset/movie.csv')
movie_genome = pd.read_csv('dataset/genome_scores.csv')

In [350]:
users = ratings['userId'].unique() #list of all users
movies = movie_props['movieId'].unique() #list of all movies

# item_features preparation

In [136]:
# Prepare table with tags relevance

genomes_dict = {}
genomes_dict['movieId'] = []

In [137]:
for i in range(len(movie_genome)):
    sample = movie_genome.iloc[i]

    if len(genomes_dict['movieId']) == 0 or genomes_dict['movieId'][-1] != sample.movieId:
        genomes_dict['movieId'].append(int(sample.movieId))
    
    tag = f'Tag_{sample.tagId}'
    if tag not in genomes_dict:
        genomes_dict[tag] = []
    
    genomes_dict[tag].append(sample.relevance)

In [237]:
movie_tag_relevance = pd.DataFrame(genomes_dict)

In [238]:
movie_tag_relevance.head()

,movieId,Tag_1.0,Tag_2.0,Tag_3.0,Tag_4.0,Tag_5.0,Tag_6.0,Tag_7.0,Tag_8.0,Tag_9.0,...,Tag_1119.0,Tag_1120.0,Tag_1121.0,Tag_1122.0,Tag_1123.0,Tag_1124.0,Tag_1125.0,Tag_1126.0,Tag_1127.0,Tag_1128.0
0,1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,...,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
1,2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,...,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
2,3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,...,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
3,4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,...,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
4,5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,...,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825


In [239]:
movie_tag_relevance.to_csv('movie_tag_relevance.csv')

In [248]:
# Prepare table with year and genres

movie_props.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [249]:
MEDIAN_YEAR = 1998
NO_GENRE = '(no genres listed)'

In [250]:
for m in movies:
    if m not in movie_props.movieId.values:
        print(m)
        movie_props = movie_props.append({'movieId': m, 'title': f'({MEDIAN_YEAR})', 'genres': NO_GENRE}, ignore_index=True)

In [251]:
# Extract year from name

def get_year(title):
    try:
        return int(title[-5:-1]) 
    except:
        try:
            return int(title[-6:-2])
        except:
            return 1998

In [252]:
years = movie_props.title.map(get_year)
movie_props['year'] = years

In [253]:
movie_props.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [254]:
# Split geners info

geners = movie_props.genres.map(lambda x: x.split('|'))

In [255]:
all_geners = []
for gs in geners.values:
    all_geners += gs
all_geners = np.unique(all_geners)

In [256]:
for g in all_geners:
    movie_props[g] = 0

In [257]:
for i, gs in enumerate(geners.values):
    for g in all_geners:
        if g in gs:
            movie_props.at[i, g] = 1

In [258]:
movie_props.head()

,movieId,title,genres,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,1995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [378]:
movie_props.shape

(27278, 22)

In [259]:
movie_props = movie_props.drop(['title', 'genres'], axis=1)

In [ ]:
# Join all info about movie

In [264]:
merged_data = pd.merge(movie_props, movie_tag_relevance, on='movieId',how='outer')

In [265]:
cols = merged_data.columns.tolist()
for c in cols:
    merged_data[c].fillna(merged_data[c].median(), inplace=True)

In [266]:
# merged_data = merged_data.drop(['movieId'], axis=1)

In [267]:
merged_data.head()

,movieId,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Tag_1119.0,Tag_1120.0,Tag_1121.0,Tag_1122.0,Tag_1123.0,Tag_1124.0,Tag_1125.0,Tag_1126.0,Tag_1127.0,Tag_1128.0
0,1,1995,0,0,0,0,0,0,0,0,...,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
1,2,1995,0,0,0,0,0,0,0,0,...,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
2,3,1995,0,0,0,0,0,0,0,0,...,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
3,4,1995,0,0,0,0,0,0,0,0,...,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
4,5,1995,0,0,0,0,0,0,0,0,...,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825


In [379]:
merged_data.shape

(27278, 1150)

In [380]:
# Along with the tag information, it works too slowly, so then I used only the genres and the year of release of the film
merged_data = movie_props

In [381]:
merged_data

,movieId,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27273,131254,2007,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27274,131256,2002,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27275,131258,2014,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27276,131260,2001,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [382]:
merged_data.to_csv('item_features.csv')

# LightFM

In [104]:
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM

In [170]:
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error, mean_absolute_error

In [400]:
class GridSearch:
    
    def __init__(self, param_grid):
        self.param_grid = param_grid
        self.best_score = {
            'auc_score': None, 
            'precision_at_k': None
        }
        self.best_params = {
            'auc_score': None, 
            'precision_at_k': None
        }
        self.best_model = {
            'auc_score': None, 
            'precision_at_k': None
        }
        
        self.param_grid['num_threads'] = 8
        self.param_grid['loss'] = 'warp'
#         self.param_grid['loss'] = 'bpr'
        self.param_grid['verbose'] = True
        self.param_grid['random_state'] = 0
        self.param_grid['item_alpha'] = 0.000005
        self.param_grid['epochs'] = 20
        
        if 'learning_rate' not in param_grid.keys():
            self.param_grid['learning_rate'] = [0.05]
        if 'no_components' not in param_grid.keys():
            self.param_grid['no_components'] = [50]
    
    
    def fit(self, data, item_features):
        self.train, self.val = random_train_test_split(data, test_percentage=0.2, random_state=self.param_grid['random_state'])
        self.item_features = item_features
        for lr in tqdm(self.param_grid['learning_rate']):
            for nc in tqdm(self.param_grid['no_components']):
                model = LightFM(
                    loss=self.param_grid['loss'],
                    random_state=self.param_grid['random_state'],
                    learning_rate=lr,
                    no_components=nc,
                    item_alpha=self.param_grid['item_alpha']
                )
                
                model = model.fit(
                    self.train,
                    item_features=item_features,
                    epochs=self.param_grid['epochs'],
                    num_threads=self.param_grid['num_threads'], 
                    verbose=self.param_grid['verbose']
                )
                
                precision_at_k = precision_at_k(model, self.val, k=10, item_features=item_features, num_threads=8).mean()
                auc_score = auc_score(model, self.val, item_features=item_features, num_threads=8).mean()
                
                if self.best_score['auc_score'] is None or self.best_score['auc_score'] > auc_score:
                    self.best_score['auc_score'] = auc_score
                    self.best_params['auc_score'] = {
                        'learning_rate': lr,
                        'no_components': nc
                    }
                    self.best_model['auc_score'] = model
                    
                if self.best_score['precision_at_k'] is None or self.best_score['precision_at_k'] > precision_at_k:
                    self.best_score['precision_at_k'] = precision_at_k
                    self.best_params['precision_at_k'] = {
                        'learning_rate': lr,
                        'no_components': nc
                    }
                    self.best_model['precision_at_k'] = model
                
                
                print(f'learning_rate: {lr} | no_components: {nc}')
                print(f'auc_score: {auc_score}')
                print(f'precision_at_k: {precision_at_k}')
                print('___________________________________________')
        print(best_score)

In [384]:
item_features = pd.read_csv('item_features.csv')

In [385]:
item_features.head()

,Unnamed: 0,movieId,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,4,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,5,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [452]:
train = pd.read_csv('mov_train.csv')

In [464]:
test = pd.read_csv('mov_test.csv')

In [451]:
test.head()

,userId,movieId,rating,timestamp
0,1,5540,4.0,2005-04-02 23:51:45
1,1,7164,3.5,2005-04-02 23:52:03
2,1,2021,4.0,2005-04-02 23:52:09
3,1,7046,4.0,2005-04-02 23:52:14
4,1,2143,4.0,2005-04-02 23:52:31


In [453]:
test.dtypes

userId         int64
movieId        int64
rating       float64
timestamp     object
dtype: object

In [454]:
movie_id_list = np.unique(train['movieId'].values.tolist() + test['movieId'].values.tolist()).tolist()
print(movie_id_list[:10])
print(movie_id_list[-10:])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[131241, 131243, 131248, 131250, 131252, 131254, 131256, 131258, 131260, 131262]


In [455]:
movie_in_train = pd.DataFrame()
movie_in_train['movieId'] = movie_id_list

In [457]:
# Different number of different films in the rating table and item features table

movie_id_in_features_list = np.unique(item_features['movieId'].values)
movie_id_in_features_list = (movie_id_in_features_list).tolist()
print(f'in features:     {len(movie_id_in_features_list)}')
print(f'in train + test: {len(movie_id_list)}')

in features:     27278
in train + test: 26737


In [458]:
print(movie_id_in_features_list[:10])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [459]:
# Remove all unnecessary movies from the table of item features

item_in_train_features = pd.merge(movie_in_train, item_features, on='movieId',how='inner')
item_in_train_features

,movieId,Unnamed: 0,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,3,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,4,1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26732,131254,27273,2007,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26733,131256,27274,2002,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26734,131258,27275,2014,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26735,131260,27276,2001,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [460]:
movies = item_in_train_features['movieId'].unique() #list of all movies in train
print(len(movies))
print(movies[:10])
print(movies[-10:])

26737
[ 1  2  3  4  5  6  7  8  9 10]
[131241 131243 131248 131250 131252 131254 131256 131258 131260 131262]


In [461]:
# Renumbering of omissions, because there are gaps

train['userId'] = train['userId'].map({u: i for i, u in enumerate(users)})
train['movieId'] = train['movieId'].map({m: i for i, m in enumerate(movies)})

In [465]:
test['userId'] = test['userId'].map({u: i for i, u in enumerate(users)})
test['movieId'] = test['movieId'].map({m: i for i, m in enumerate(movies)})

In [466]:
train

,userId,movieId,rating,timestamp
0,0,907,3.5,2004-09-10 03:06:38
1,0,902,3.5,2004-09-10 03:07:01
2,0,2597,3.5,2004-09-10 03:07:30
3,0,1532,3.5,2004-09-10 03:07:36
4,0,1057,4.0,2004-09-10 03:07:45
...,...,...,...,...
15776687,138492,1871,4.0,2009-10-28 17:32:34
15776688,138492,752,4.0,2009-10-29 22:19:21
15776689,138492,5686,3.5,2009-10-29 22:19:35
15776690,138492,1395,4.0,2009-10-29 22:20:05


In [467]:
test

,userId,movieId,rating,timestamp
0,0,5443,4.0,2005-04-02 23:51:45
1,0,7052,3.5,2005-04-02 23:52:03
2,0,1937,4.0,2005-04-02 23:52:09
3,0,6934,4.0,2005-04-02 23:52:14
4,0,2059,4.0,2005-04-02 23:52:31
...,...,...,...,...
4086023,138492,6424,3.0,2009-12-07 18:18:28
4086024,138492,11901,4.0,2009-12-07 18:18:40
4086025,138492,1247,3.0,2010-01-01 20:42:32
4086026,138492,6884,3.0,2010-01-01 20:42:35


In [468]:
print(f'len: {len(np.unique(train["movieId"].values))}')
print(f'max: {np.max(train["movieId"].values)}')
print(f'min: {np.min(train["movieId"].values)}')

len: 22057
max: 26731
min: 0


In [469]:
# Prepare interaction sparce matrix

rows = []
cols = []
data = []

for i, row in tqdm(train.iterrows()):    
    rows.append(row.userId)
    cols.append(row.movieId)
    data.append(row.rating)

15776692it [23:05, 11386.36it/s]


In [470]:
print(np.max(cols))
print(np.min(cols))

26731
0


In [471]:
train_coo = sparse.coo_matrix((data, (rows, cols)), shape=(np.max(rows) + 1, np.max(cols) + 1))

In [472]:
len(np.unique(cols))

22057

In [473]:
len(np.unique(rows))

138493

In [474]:
train_coo.shape

(138493, 26732)

In [475]:
scipy.sparse.save_npz('sparse_train.npz', train_coo)

In [476]:
train_sparse_matrix = scipy.sparse.load_npz('sparse_train.npz')

In [477]:
item_in_train_features = item_in_train_features.drop(['movieId'], axis=1)

In [478]:
item_in_train_features.shape

(26737, 22)

In [479]:
item_features_sparse = scipy.sparse.csr_matrix(item_in_train_features.values)

In [480]:
item_features_sparse.shape

(26737, 22)

In [482]:
usedId = []
movieId = []
y_true = [] # true rating

for i, row in tqdm(test.iterrows()):    
    usedId.append(row.userId)
    movieId.append(row.movieId)
    y_true.append(row.rating)

4086028it [06:01, 11305.36it/s]


In [516]:
test_sparse = sparse.coo_matrix((y_true, (usedId, movieId)), shape=(np.max(usedId) + 1, np.max(movieId) + 1))

In [523]:
test_sparse.shape

(138493, 26737)

In [534]:
model = LightFM(
    loss='warp',
    random_state=0,
    learning_rate=0.01,
    no_components=50,
    item_alpha=0.000005
)
    
model = model.fit(
    train_sparse_matrix,
    item_features=item_features_sparse,
    epochs=20,
    num_threads=8, 
    verbose=True
)

Epoch: 100%|██████████| 20/20 [14:09<00:00, 42.50s/it]


In [535]:
# gs = GridSearch(param_grid={
#     'learning_rate': [0.001, 0.01, 0.05, 0.1],
#     'no_components': [5, 10, 30, 50]
# })
# gs.fit(train_sparse_matrix, item_features_sparse)

In [536]:
train_precision = precision_at_k(model, train_sparse_matrix, k=10, item_features=item_features_sparse, num_threads=8).mean()

In [537]:
test_precision = precision_at_k(model, test_sparse, k=10, item_features=item_features_sparse, num_threads=8).mean()

In [538]:
train_auc = auc_score(model, train_sparse_matrix, item_features=item_features_sparse, num_threads=8).mean()

In [539]:
test_auc = auc_score(model, test_sparse, item_features=item_features_sparse, num_threads=8).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.10, test 0.02.
AUC: train 0.90, test 0.87.


In [540]:
train_recall = recall_at_k(model, train_sparse_matrix, k=10, item_features=item_features_sparse, num_threads=8).mean()

In [541]:
test_recall = recall_at_k(model, test_sparse, k=10, item_features=item_features_sparse, num_threads=8).mean()

In [542]:
print('Recall: train %.2f, test %.2f.' % (train_recall, test_recall))

Recall: train 0.02, test 0.01.


In [543]:
model = LightFM(
    loss='bpr',
    random_state=0,
    learning_rate=0.01,
    no_components=50,
    item_alpha=0.000005
)
    
model = model.fit(
    train_sparse_matrix,
    item_features=item_features_sparse,
    epochs=20,
    num_threads=8, 
    verbose=True
)

Epoch: 100%|██████████| 20/20 [24:26<00:00, 73.34s/it]


In [544]:
train_precision = precision_at_k(model, train_sparse_matrix, k=10, item_features=item_features_sparse, num_threads=8).mean()

In [545]:
test_precision = precision_at_k(model, test_sparse, k=10, item_features=item_features_sparse, num_threads=8).mean()

In [546]:
train_auc = auc_score(model, train_sparse_matrix, item_features=item_features_sparse, num_threads=8).mean()

In [547]:
test_auc = auc_score(model, test_sparse, item_features=item_features_sparse, num_threads=8).mean()

In [548]:
train_recall = recall_at_k(model, train_sparse_matrix, k=10, item_features=item_features_sparse, num_threads=8).mean()

In [549]:
test_recall = recall_at_k(model, test_sparse, k=10, item_features=item_features_sparse, num_threads=8).mean()

In [550]:
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
print('Recall: train %.2f, test %.2f.' % (train_recall, test_recall))

Precision: train 0.03, test 0.01.
AUC: train 0.84, test 0.83.
Recall: train 0.01, test 0.00.
